# 1. ดาวน์โหลด library

In [ ]:
# รันเฉพาะบน Colab เพื่อให้ติดตั้งไลบรารีแบบไม่ดึง dependencies ใหญ่ ๆ เข้ามาทั้งหมด
%%bash
pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl==0.15.2 triton cut_cross_entropy unsloth_zoo
pip install sentencepiece protobuf "datasets>=3.4.1" huggingface_hub hf_transfer
pip install transformers==4.51.3
pip install --no-deps unsloth
pip install pandas


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.4/43.4 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.8/145.8 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 20.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 98.8 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.52.3
    Uninstalling transforme

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
unsloth-zoo 2025.5.11 requires msgspec, which is not installed.
unsloth-zoo 2025.5.11 requires tyro, which is not installed.
unsloth-zoo 2025.5.11 requires protobuf<4.0.0, but you have protobuf 5.29.5 which is incompatible.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-nvrtc-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have n

# 2. โหลดและเตรียมข้อมูล (preprocessing)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
# อ่านไฟล์ esg-score.csv
df_esg = pd.read_csv("/content/drive/MyDrive/dataset/esg-score.csv")
print("ESG dataframe shape:", df_esg.shape)
print(df_esg.head())

ESG dataframe shape: (265, 14)
  ชื่อย่อ                                  Company Name  \
0     AAV          ASIA AVIATION PUBLIC COMPANY LIMITED   
1     ACE  ABSOLUTE CLEAN ENERGY PUBLIC COMPANY LIMITED   
2     ADB             APPLIED DB PUBLIC COMPANY LIMITED   
3  ADVANC  ADVANCED INFO SERVICE PUBLIC COMPANY LIMITED   
4     AGE      ASIA GREEN ENERGY PUBLIC COMPANY LIMITED   

                                         ชื่อบริษัท กลุ่มอุตสาหกรรม 2024 2023  \
0            บริษัท เอเชีย เอวิเอชั่น จำกัด (มหาชน)        Services  NaN  NaN   
1  บริษัท แอ๊บโซลูท คลีน เอ็นเนอร์จี้ จำกัด (มหาชน)       Resources  AAA   AA   
2                บริษัท แอ็พพลาย ดีบี จำกัด (มหาชน)     Industrials   AA    A   
3    บริษัท แอดวานซ์ อินโฟร์ เซอร์วิส จำกัด (มหาชน)      Technology   AA  AAA   
4         บริษัท เอเชีย กรีน เอนเนอจี จำกัด (มหาชน)       Resources  NaN  BBB   

  2022 2021 2020 2019 2018 2017 2016 2015  
0  Yes  Yes  Yes  Yes  NaN  NaN  NaN  NaN  
1  Yes  NaN  NaN  NaN  NaN  NaN  NaN  N

In [ ]:
# อ่านไฟล์ news.csv
df_news = pd.read_csv("/content/drive/MyDrive/dataset/news.csv")
print("News dataframe shape:", df_news.shape)
print(df_news.head())

News dataframe shape: (1117, 6)
  ตัวย่อหุ้น                                         หัวข้อข่าว  \
0        WHA  สรุป pp Day : WHA จ่อปิด 2 ดีลใหญ่ ช่วง Q2Q3 น...   
1        WHA  สะกิดต่อมหุ้น อัปเดต ประจำวันที่ 30 พ.ค. 2568:...   
2        WHA  WHA จับตาผลภาษีสหรัฐฯ ชัดเจนในครึ่งปีหลัง ก่อน...   
3        WHA  WHA กำไร Q1/68 ที่ 2,075.47 ลบ. โต 52.06% ขายท...   
4        WHA  หุ้นนิคมฯ ไตรมาส 2/68 ยอดขายแข็งแกร่ง ด้านภาษี...   

  วันที่เขียนข่าว                                        เนื้อหาข่าว  \
0      30/05/2025  ข้อมูลจาก pp Day ระบุว่า WHA กำลังเจรจาปิดดีลล...   
1      30/05/2025  ในการอัปเดตตลาดหุ้นประจำวัน WHA ปรับตัวขึ้นกว่...   
2      02/06/2025  สำนักข่าวรายงานว่า WHA กำลังจับตาการกำหนดอัตรา...   
3      12/05/2025  WHA รายงานกำไรสุทธิไตรมาส 1/2568 ที่ 2,075.47 ...   
4      02/06/2025  บทวิเคราะห์จาก สำนักข่าวบางกอกบิสเนส รายงานว่า...   

  วิเคราะห์                                             เหตุผล  
0       บวก  การมีลูกค้าดาต้าเซ็นเตอร์ต่อคิวรอทำสัญญาถึงกว่...  
1 

In [ ]:
# ดู dtype เดิมของคอลัมน์
print("ก่อนแปลง dtype:", df_news["วันที่เขียนข่าว"].dtype)

ก่อนแปลง dtype: object


In [ ]:
# แปลงเป็น datetime (format: วัน/เดือน/ปี)
df_news["วันที่เขียนข่าว"] = pd.to_datetime(
    df_news["วันที่เขียนข่าว"],
    dayfirst=True,         # เพราะ format ที่ให้มาในคือ DD/MM/YYYY
    errors="coerce"        # ถ้าแปลงไม่ได้ จะได้ NaT แทน
)

In [ ]:
# ตรวจสอบ dtype หลังแปลง
print("หลังแปลง dtype:", df_news["วันที่เขียนข่าว"].dtype)

หลังแปลง dtype: datetime64[ns]


# 3. รวมข้อมูล ESG กับข่าว (merge)

In [ ]:
# หากชื่อคอลัมน์ต่างกัน ให้ปรับให้ตรงกันก่อน merge
df_esg = df_esg.rename(columns={"ชื่อย่อ": "ตัวย่อหุ้น"})

# Merge โดยใช้ left join (ให้เอาทุกบรรทัดใน df_news เป็นหลัก)
df_merged = pd.merge(df_news, df_esg, on="ตัวย่อหุ้น", how="left")

print("After merge:", df_merged.shape)
df_merged.head()


After merge: (1117, 19)


,ตัวย่อหุ้น,หัวข้อข่าว,วันที่เขียนข่าว,เนื้อหาข่าว,วิเคราะห์,เหตุผล,Company Name,ชื่อบริษัท,กลุ่มอุตสาหกรรม,2024,2023,2022,2021,2020,2019,2018,2017,2016,2015
0,WHA,สรุป pp Day : WHA จ่อปิด 2 ดีลใหญ่ ช่วง Q2Q3 น...,2025-05-30,ข้อมูลจาก pp Day ระบุว่า WHA กำลังเจรจาปิดดีลล...,บวก,การมีลูกค้าดาต้าเซ็นเตอร์ต่อคิวรอทำสัญญาถึงกว่...,WHA CORPORATION PUBLIC COMPANY LIMITED,บริษัท ดับบลิวเอชเอ คอร์ปอเรชั่น จำกัด (มหาชน),Property & Construction,AAA,AAA,Yes,Yes,Yes,NaN,NaN,NaN,NaN,NaN
1,WHA,สะกิดต่อมหุ้น อัปเดต ประจำวันที่ 30 พ.ค. 2568:...,2025-05-30,ในการอัปเดตตลาดหุ้นประจำวัน WHA ปรับตัวขึ้นกว่...,บวก,การเปลี่ยนแปลงเชิงบวกในกรอบนโยบายภาษีสหรัฐฯ จะ...,WHA CORPORATION PUBLIC COMPANY LIMITED,บริษัท ดับบลิวเอชเอ คอร์ปอเรชั่น จำกัด (มหาชน),Property & Construction,AAA,AAA,Yes,Yes,Yes,NaN,NaN,NaN,NaN,NaN
2,WHA,WHA จับตาผลภาษีสหรัฐฯ ชัดเจนในครึ่งปีหลัง ก่อน...,2025-06-02,สำนักข่าวรายงานว่า WHA กำลังจับตาการกำหนดอัตรา...,บวก,การจับตาภาษีนำเข้าชัดเจนจะช่วยให้ WHA สามารถทบ...,WHA CORPORATION PUBLIC COMPANY LIMITED,บริษัท ดับบลิวเอชเอ คอร์ปอเรชั่น จำกัด (มหาชน),Property & Construction,AAA,AAA,Yes,Yes,Yes,NaN,NaN,NaN,NaN,NaN
3,WHA,"WHA กำไร Q1/68 ที่ 2,075.47 ลบ. โต 52.06% ขายท...",2025-05-12,"WHA รายงานกำไรสุทธิไตรมาส 1/2568 ที่ 2,075.47 ...",บวก,กำไรสุทธิที่เติบโตสูงถึง 52.06% ในไตรมาสแรกเป็...,WHA CORPORATION PUBLIC COMPANY LIMITED,บริษัท ดับบลิวเอชเอ คอร์ปอเรชั่น จำกัด (มหาชน),Property & Construction,AAA,AAA,Yes,Yes,Yes,NaN,NaN,NaN,NaN,NaN
4,WHA,หุ้นนิคมฯ ไตรมาส 2/68 ยอดขายแข็งแกร่ง ด้านภาษี...,2025-06-02,บทวิเคราะห์จาก สำนักข่าวบางกอกบิสเนส รายงานว่า...,บวก,การเข้าร่วมเจรจาผ่อนปรนภาษีช่วยลดต้นทุนตลอดห่ว...,WHA CORPORATION PUBLIC COMPANY LIMITED,บริษัท ดับบลิวเอชเอ คอร์ปอเรชั่น จำกัด (มหาชน),Property & Construction,AAA,AAA,Yes,Yes,Yes,NaN,NaN,NaN,NaN,NaN


In [ ]:
# กำหนดรายชื่อคอลัมน์ปีที่ต้องการตรวจสอบ
year_columns = ['2024', '2023', '2022', '2021', '2020', '2019', '2018', '2017', '2016', '2015']

# วนลูปเพื่อแทนที่ค่า NaN ด้วย 'no rating' ในคอลัมน์ที่กำหนด
for col in year_columns:
  if col in df_merged.columns: # ตรวจสอบว่าคอลัมน์มีอยู่ใน DataFrame จริงหรือไม่
    df_merged[col] = df_merged[col].fillna('no rating')

print("DataFrame after filling NaN values:")
display(df_merged.head())

DataFrame after filling NaN values:


,ตัวย่อหุ้น,หัวข้อข่าว,วันที่เขียนข่าว,เนื้อหาข่าว,วิเคราะห์,เหตุผล,Company Name,ชื่อบริษัท,กลุ่มอุตสาหกรรม,2024,2023,2022,2021,2020,2019,2018,2017,2016,2015
0,WHA,สรุป pp Day : WHA จ่อปิด 2 ดีลใหญ่ ช่วง Q2Q3 น...,2025-05-30,ข้อมูลจาก pp Day ระบุว่า WHA กำลังเจรจาปิดดีลล...,บวก,การมีลูกค้าดาต้าเซ็นเตอร์ต่อคิวรอทำสัญญาถึงกว่...,WHA CORPORATION PUBLIC COMPANY LIMITED,บริษัท ดับบลิวเอชเอ คอร์ปอเรชั่น จำกัด (มหาชน),Property & Construction,AAA,AAA,Yes,Yes,Yes,no rating,no rating,no rating,no rating,no rating
1,WHA,สะกิดต่อมหุ้น อัปเดต ประจำวันที่ 30 พ.ค. 2568:...,2025-05-30,ในการอัปเดตตลาดหุ้นประจำวัน WHA ปรับตัวขึ้นกว่...,บวก,การเปลี่ยนแปลงเชิงบวกในกรอบนโยบายภาษีสหรัฐฯ จะ...,WHA CORPORATION PUBLIC COMPANY LIMITED,บริษัท ดับบลิวเอชเอ คอร์ปอเรชั่น จำกัด (มหาชน),Property & Construction,AAA,AAA,Yes,Yes,Yes,no rating,no rating,no rating,no rating,no rating
2,WHA,WHA จับตาผลภาษีสหรัฐฯ ชัดเจนในครึ่งปีหลัง ก่อน...,2025-06-02,สำนักข่าวรายงานว่า WHA กำลังจับตาการกำหนดอัตรา...,บวก,การจับตาภาษีนำเข้าชัดเจนจะช่วยให้ WHA สามารถทบ...,WHA CORPORATION PUBLIC COMPANY LIMITED,บริษัท ดับบลิวเอชเอ คอร์ปอเรชั่น จำกัด (มหาชน),Property & Construction,AAA,AAA,Yes,Yes,Yes,no rating,no rating,no rating,no rating,no rating
3,WHA,"WHA กำไร Q1/68 ที่ 2,075.47 ลบ. โต 52.06% ขายท...",2025-05-12,"WHA รายงานกำไรสุทธิไตรมาส 1/2568 ที่ 2,075.47 ...",บวก,กำไรสุทธิที่เติบโตสูงถึง 52.06% ในไตรมาสแรกเป็...,WHA CORPORATION PUBLIC COMPANY LIMITED,บริษัท ดับบลิวเอชเอ คอร์ปอเรชั่น จำกัด (มหาชน),Property & Construction,AAA,AAA,Yes,Yes,Yes,no rating,no rating,no rating,no rating,no rating
4,WHA,หุ้นนิคมฯ ไตรมาส 2/68 ยอดขายแข็งแกร่ง ด้านภาษี...,2025-06-02,บทวิเคราะห์จาก สำนักข่าวบางกอกบิสเนส รายงานว่า...,บวก,การเข้าร่วมเจรจาผ่อนปรนภาษีช่วยลดต้นทุนตลอดห่ว...,WHA CORPORATION PUBLIC COMPANY LIMITED,บริษัท ดับบลิวเอชเอ คอร์ปอเรชั่น จำกัด (มหาชน),Property & Construction,AAA,AAA,Yes,Yes,Yes,no rating,no rating,no rating,no rating,no rating


# 4. สร้างรูปแบบข้อมูล (prompt-response) สำหรับ Fine‐tuning

In [ ]:
# ฟังก์ชันช่วยสรุป ESG Scores
def summarize_esg_scores(row, years_range=range(2015, 2025)):
    parts = []
    for y in years_range:
        val = row.get(str(y), None)
        if pd.isna(val) or val == "":
            continue
        parts.append(f"{y}: {val}")
    if not parts:
        return "ไม่มีข้อมูล ESG Score"
    return ", ".join(parts)

In [ ]:
# สร้าง list ของ conversations
records = []
for idx, row in df_merged.iterrows():
    ticker       = row["ตัวย่อหุ้น"]
    company_en   = row.get("Company Name", "")
    company_th   = row.get("ชื่อบริษัท", "")
    industry     = row.get("กลุ่มอุตสาหกรรม", "")
    esg_summary  = summarize_esg_scores(row)

    # ** ส่วนใหญ่ตรงนี้เกิด error เพราะเป็น string **
    # เราจึงเช็คว่าเป็น datetime หรือไม่ก่อนใช้ strftime
    raw_date = row["วันที่เขียนข่าว"]
    if pd.isna(raw_date):
        news_date = ""
    elif isinstance(raw_date, pd.Timestamp):
        news_date = raw_date.strftime("%d/%m/%Y")
    else:
        # ถ้าไม่ใช่ Timestamp แล้ว (อาจเป็น string หรือ object), ให้ใช้เป็น string ตรง ๆ
        news_date = str(raw_date)

    news_title   = row["หัวข้อข่าว"]
    news_content = row["เนื้อหาข่าว"]

    # ตัดกรณีวิเคราะห์/เหตุผล ถ้า null หรือ NaN
    analysis_text = row["วิเคราะห์"] if pd.notna(row["วิเคราะห์"]) else ""
    reason_text   = row["เหตุผล"] if pd.notna(row["เหตุผล"]) else ""
    answer_text   = analysis_text
    if reason_text.strip():
        answer_text += "\n\nเหตุผล: " + reason_text

    # สร้าง prompt (user message)
    user_prompt = (
        f"หุ้น {ticker} ({company_en})\n"
        f"ชื่อบริษัท: {company_th}\n"
        f"กลุ่มอุตสาหกรรม: {industry}\n"
        f"ESG Scores: {esg_summary}\n"
        f"ข่าวประจำวันที่ {news_date}: {news_title}\n"
        f"{news_content}\n\n"
        "กรุณาวิเคราะห์สถานการณ์หุ้นนี้ โดยอ้างอิงจาก ESG Rating และเนื้อหาข่าว"
    )

    records.append({
        "conversations": [
            {"role": "user",    "content": user_prompt},
            {"role": "assistant", "content": answer_text}
        ]
    })

In [ ]:
# สร้าง HuggingFace Dataset
from datasets import Dataset
hf_dataset = Dataset.from_list(records)
print("จำนวนตัวอย่างใน hf_dataset =", len(hf_dataset))

จำนวนตัวอย่างใน hf_dataset = 1117


# 5. เตรียมโมเดล UnsLoth + PEFT (LoRA) จาก UnsLoth

In [ ]:
import torch
from unsloth import FastLanguageModel, is_bfloat16_supported
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth.chat_templates import get_chat_template, standardize_sharegpt, train_on_responses_only

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [ ]:
# กำหนดพารามิเตอร์เบื้องต้น สำหรับโหลดโมเดล
max_seq_length = 2048
dtype = None  # ให้ระบบเลือก dtype อัตโนมัติ (เท่าที่ Colab T4 รองรับจะเป็น float16)
load_in_4bit = True  # ใช้ 4bit quantization เพื่อลด VRAM


In [ ]:
# โหลดโมเดล + Tokenizer
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Llama-3.2-3B-Instruct-bnb-4bit",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
    # token="hf_..."  # กรณีโมเดลต้องใช้ token จาก HuggingFace ให้ใส่ตรงนี้
)

==((====))==  Unsloth 2025.5.9: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.7k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

chat_template.jinja:   0%|          | 0.00/3.83k [00:00<?, ?B/s]

In [ ]:
# ทำ PEFT (LoRA) ให้กับโมเดล
model = FastLanguageModel.get_peft_model(
    model,
    r=16,  # rank สำหรับ LoRA (ปรับได้ 8, 16, 32, 64 ฯลฯ)
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                    "gate_proj", "up_proj", "down_proj"],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407,
    use_rslora=False,
    loftq_config=None,
)


Unsloth 2025.5.9 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [ ]:
# โหลด Chat Template ของ UnsLoth (รูปแบบเดียวกับ llama-chat)
tokenizer = get_chat_template(
    tokenizer,
    chat_template="llama-3.1"
)

# 6. แปลง Dataset ให้เป็นฟอร์แมต Text + SFTTrainer

In [ ]:
# ฟังก์ชันสำหรับแปลง list[{"role":…}] ให้เป็น string ที่ tokenizer เข้าใจ
def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize=False, add_generation_prompt=False) for convo in convos]
    # เราจะส่ง field ใหม่ชื่อ "text" ให้ SFTTrainer ใช้ fine-tune
    return {"text": texts}

In [ ]:
# เรียก map เพื่อเพิ่ม field "text"
hf_dataset = hf_dataset.map(formatting_prompts_func, batched=True, remove_columns=["conversations"])
# ตรวจสอบว่ามี field "text" แล้ว
print(hf_dataset.column_names)

Map:   0%|          | 0/1117 [00:00<?, ? examples/s]

['text']


# 7. สร้าง SFTTrainer แล้วเริ่ม Fine‐tune

In [ ]:
# สร้าง DataCollator ที่เหมาะกับ Seq2Seq (แต่สำหรับ Chat SFT เราอาจใช้ DataCollatorForSeq2Seq ได้เลย)
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, pad_to_multiple_of=8)

In [ ]:
# สร้าง SFTTrainer
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=hf_dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    data_collator=data_collator,
    dataset_num_proc=2,      # จำนวน process สำหรับ map (ปรับลดได้หากไม่มี CPU เยอะ)
    packing=False,           # packing ช่วยให้ train เร็วขึ้นแต่ไส้กรอกสลับข้อความ อาจไม่จำเป็นสำหรับ chatbot
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=10,
        max_steps=200,       # ปรับตามขนาด dataset และงบ VRAM (T4: ประมาณ 200–500 step)
        learning_rate=2e-4,
        fp16=not is_bfloat16_supported(),   # ใช้ fp16 บน T4 ได้
        bf16=is_bfloat16_supported(),
        logging_steps=10,
        optim="adamw_8bit",  # ใช้ AdamW 8-bit เพื่อลด VRAM
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs_chatbot_esg",
        report_to="none",    # ปิด logging กับ WandB
    ),
)

Unsloth: Tokenizing ["text"]:   0%|          | 0/1117 [00:00<?, ? examples/s]

In [ ]:
# กำหนดให้ Trainer ฝึกแค่ “ตอบ” ของ assistant (จะช่วยลด noise จาก prompt)
trainer = train_on_responses_only(
    trainer,
    instruction_part="<|start_header_id|>user<|end_header_id|>\n\n",
    response_part="<|start_header_id|>assistant<|end_header_id|>\n\n",
)

Map (num_proc=2):   0%|          | 0/1117 [00:00<?, ? examples/s]

In [ ]:
# เริ่ม train
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,117 | Num Epochs = 2 | Total steps = 200
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 24,313,856/3,000,000,000 (0.81% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
10,2.171900
20,1.744100
30,1.551700
40,1.493400
50,1.419800
60,1.413700
70,1.370300
80,1.368800
90,1.301600
100,1.325000


In [ ]:
# บันทึกโมเดลเป็น GGUF แบบ 4-bit k_m (q4_k_m)
model.save_pretrained_gguf(
    "/content/drive/MyDrive/model",
    tokenizer,
    quantization_method="q4_k_m"
)


Unsloth: You have 1 CPUs. Using `safe_serialization` is 10x slower.
We shall switch to Pytorch saving, which might take 3 minutes and not 30 minutes.
To force `safe_serialization`, set it to `None` instead.
Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 2.2G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 5.38 out of 12.67 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


100%|██████████| 28/28 [00:01<00:00, 16.80it/s]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving /content/drive/MyDrive/model/pytorch_model-00001-of-00002.bin...
Unsloth: Saving /content/drive/MyDrive/model/pytorch_model-00002-of-00002.bin...
Done.


Unsloth: Converting llama model. Can use fast conversion = False.


==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits might take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['q4_k_m'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: Installing llama.cpp. This might take 3 minutes...
Unsloth: CMAKE detected. Finalizing some steps for installation.
Unsloth: [1] Converting model at /content/drive/MyDrive/model into f16 GGUF format.
The output location will be /content/drive/MyDrive/model/unsloth.F16.gguf
This might take 3 minutes...
INFO:hf-to-gguf:Loading model: model
INFO:hf-to-gguf:Model architecture: LlamaForCausalLM
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:rope_freqs.weight,           torch.float32 --> F32, shape = {64}
INFO:hf-to-gguf:gguf: loading model weight map from 'pytorch_model.

In [ ]:
# ติดตั้ง dependencies
!apt-get update -y
!apt-get install -y build-essential cmake git

# Clone llama.cpp
%cd /content
!git clone https://github.com/ggerganov/llama.cpp.git
%cd llama.cpp

# Build
!mkdir -p build
%cd build
!cmake ..
!make -j$(nproc)


Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,741 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,245 kB]
Get:13 https://ppa.launchpad

In [ ]:
# .gguf ถูกอัปโหลดไปที่โฟลเดอร์นี้
!ls -lh "/content/drive/MyDrive/model/"


total 14G
-rw------- 1 root root  922 Jun  5 07:12 config.json
-rw------- 1 root root  234 Jun  5 07:12 generation_config.json
-rw------- 1 root root 1.9K Jun  5 07:25 Modelfile
-rw------- 1 root root 4.7G Jun  5 07:13 pytorch_model-00001-of-00002.bin
-rw------- 1 root root 1.4G Jun  5 07:13 pytorch_model-00002-of-00002.bin
-rw------- 1 root root  21K Jun  5 07:13 pytorch_model.bin.index.json
-rw------- 1 root root  454 Jun  5 07:12 special_tokens_map.json
-rw------- 1 root root  55K Jun  5 07:12 tokenizer_config.json
-rw------- 1 root root  17M Jun  5 07:12 tokenizer.json
-rw------- 1 root root 6.0G Jun  5 07:18 unsloth.F16.gguf
-rw------- 1 root root 1.9G Jun  5 07:25 unsloth.Q4_K_M.gguf


In [ ]:
%cd /content/llama.cpp/build/bin

MODEL_GGUF="/content/drive/MyDrive/model/unsloth.Q4_K_M.gguf"

# ตรวจสอบไฟล์
!ls -lh "$MODEL_GGUF"



/content/llama.cpp/build/bin
-rw------- 1 root root 1.9G Jun  5 07:25 /content/drive/MyDrive/model/unsloth.Q4_K_M.gguf


In [ ]:
# รัน generate
!./llama-cli \
    -m "$MODEL_GGUF" \
    -p "AMATA-WHA ร่วงหนักรับ “ทรัมป์” เก็บภาษีตอบโต้ไทย-เวียดนามสูงห่วงกระทบย้ายฐานผลิต" \
    -n 128

build: 5592 (9e31bec4) with cc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0 for x86_64-linux-gnu
main: llama backend init
main: load the model and apply lora adapter, if any
llama_model_loader: loaded meta data with 28 key-value pairs and 255 tensors from /content/drive/MyDrive/model/unsloth.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Model
llama_model_loader: - kv   3:                         general.size_label str              = 3.2B
llama_model_loader: - kv   4:                          llama.block_count u32              = 28
llama_model_loader: - kv   5:                       llama.context_length u32              =